In [1]:
#!pip install fredapi

In [2]:
import pandas as pd
import numpy as np
from fredapi import Fred
from tqdm import tqdm

from typing import Any

In [3]:
import getpass
api_key = getpass.getpass(prompt='Please enter your FRED API Key: ')

Please enter your FRED API Key: ········


In [4]:
# Function to load series data using Fred API. 
# Inputs:
#    series_id : ID of series rto load (e.g. UNRATE for unemployment rate, CPIAUSCL for CPI)
#    value_col_name: name of value column. This is used to name the column that will contain
#                    values for the series
def fred_series(series_id : str, value_col_name = 'VALUE') -> pd.DataFrame:
    fred = Fred(api_key=api_key)
    raw_data = fred.get_series(series_id)
    
    # reset index - this will generate an index for the dataframe. And it will move the YYYY-MM-DD
    # index to a column named 'index'. In addition, the value column name will be 0
    data = raw_data.reset_index()
    
    # Rename columns
    data = data.rename(columns={"index":"DATE", 0:value_col_name})
    
    # Round values to 2 decimal places
    data[value_col_name] = round(data[value_col_name], 2)
    return data

In [5]:
from collections import namedtuple

# Series - namedtuple containing series id, name of target value column, csv file name
SeriesSource = namedtuple('SeriesSource', ['id', 'value_col', 'file_name'])

# SeriesData - namedtuple containing series and data
SeriesData = namedtuple('SeriesData', ['series', 'data'])


In [6]:
# Define series data sources
source = [
           SeriesSource(id='FEDFUNDS', value_col='EFFR', file_name='effr.csv'),
    
           SeriesSource(id='UNRATE', value_col='UNEMPLOYMENT', file_name='unrate.csv'),
    
           SeriesSource(id='CPIAUCSL', value_col='CPI', file_name='cpi.csv'),

           SeriesSource(id='DPCCRV1Q225SBEA', value_col='PCE', file_name='pce.csv'),

           SeriesSource(id='FPCPITOTLZGUSA', value_col='INFLATION', file_name='inflation.csv')
         ]
source

[SeriesSource(id='FEDFUNDS', value_col='EFFR', file_name='effr.csv'),
 SeriesSource(id='UNRATE', value_col='UNEMPLOYMENT', file_name='unrate.csv'),
 SeriesSource(id='CPIAUCSL', value_col='CPI', file_name='cpi.csv'),
 SeriesSource(id='DPCCRV1Q225SBEA', value_col='PCE', file_name='pce.csv'),
 SeriesSource(id='FPCPITOTLZGUSA', value_col='INFLATION', file_name='inflation.csv')]

In [7]:
# Fetch series data
target = []
for src in tqdm(source, "Fetching series data"):
    data = fred_series(src.id, src.value_col)
    target.append(SeriesData(series=src, data=data))
 

Fetching series data: 100%|███████████████████████| 5/5 [00:01<00:00,  2.81it/s]


In [8]:
# Persist series data
for seriesData in tqdm(target, "Saving series data"):
    seriesData.data.to_csv(seriesData.series.file_name, index=False)

Saving series data: 100%|████████████████████████| 5/5 [00:00<00:00, 211.79it/s]


In [9]:
target[1]

SeriesData(series=SeriesSource(id='UNRATE', value_col='UNEMPLOYMENT', file_name='unrate.csv'), data=          DATE  UNEMPLOYMENT
0   1948-01-01           3.4
1   1948-02-01           3.8
2   1948-03-01           4.0
3   1948-04-01           3.9
4   1948-05-01           3.5
..         ...           ...
903 2023-04-01           3.4
904 2023-05-01           3.7
905 2023-06-01           3.6
906 2023-07-01           3.5
907 2023-08-01           3.8

[908 rows x 2 columns])